In [1]:
import numpy as np
import pandas as pd
import math
import time
from sklearn.model_selection import train_test_split

In [2]:
rdata = pd.read_csv('sample_treino.csv') #read the data

data = rdata.iloc[:,0:len(rdata.columns)-1] # set headers Cálculo1,LPT,P1,IC,Cálculo2,cra 
data = np.c_[np.ones(len(rdata)),data]      #add extra columns with ones

Y = rdata.iloc[:,len(rdata.columns)-1:len(rdata.columns)].values #saving the Y the cra
w = np.zeros([1,len(rdata.columns)]) #the variables

In [7]:
data_train, data_test, Y_train, Y_test = train_test_split(data, Y, test_size=0.2, random_state=0)

Error function

$ MSE(w) = \frac{1}{N} \sum_{i=1}^{N} {(X^T \cdot W - Y_i)^2}$

In [9]:
def MSE(X,Y,w):
    return np.sum(np.power(((X @ w.T)-Y),2))/len(X)

In [11]:
MSE(data_train,Y_train,w)

55.13513034200704

$ w^{t+1} = w^t -2H^t (y-H_w)$

$\nabla RSS(w) = \nabla [(y - Hw)^T (y - Hw)]$

$\nabla RSS(w) = -2H^T (y - Hw)$

In [15]:
def grad_rss(w,X,Y):
    return X.T @ ( (X @ w.flatten()) - Y.flatten() )/len(X)

$ w^{t+1} = w^t + \gamma * \nabla RSS(w) $

In [16]:
def step_gradient(w,X,Y,learningRate):
    #return w - ((learningRate/len(X)) * np.sum(X * (X @ w.T - Y), axis=0) )
    return w - ((learningRate) * grad_rss(w,X,Y) )

In [17]:
def gradient_descent_runner(starting_w,X,Y, learning_rate, epsilon):
    w = starting_w
    grad = epsilon + 1
    i = 0
    while (np.linalg.norm(grad)>=epsilon):
        nw = step_gradient(w, X, Y, learning_rate)
        #print('{:.3f}'.format(np.linalg.norm(grad)),np.linalg.norm(grad)>=epsilon)
        grad = grad_rss(nw,X, Y,)
        if (np.linalg.norm(grad) != np.inf):
            w = nw
        else:
            return w,i
        i+= 1
    return w,i

sss

In [29]:
tic = time.time()
model,i = gradient_descent_runner(w,data_train,Y_train,0.001,0.01)
toc = time.time()

print("Gradiente descendente convergiu com o modelo ({0}) em {1} iterações, com  error = {2}".format(model,i, MSE(data_train,Y_train,model)))
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")

Gradiente descendente convergiu com o modelo ([[ 0.72492234  0.08819466  0.09777953  0.19481522  0.46942697 -0.0149916 ]]) em 55995 iterações, com  error = 0.3972166027997065
Versão vetorizada rodou em: 2435.608148574829 ms


sdsds

In [21]:
MSE(data_test,Y_test,model)

0.5484769039402049

## sklearn version

In [34]:
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
tic = time.time()
regressor.fit(data_train, Y_train)
toc = time.time()

print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")
regressor.coef_

Versão vetorizada rodou em: 11.993169784545898 ms


array([[ 0.        ,  0.05880505,  0.0398723 ,  0.19673024,  0.39718294,
        -0.02694486]])

In [37]:
#coeff_df = pd.DataFrame(regressor.coef_, rdata.columns[:-1], columns=['Coefficient'])  
#coeff_df  
#y_pred = regressor.predict(X_test)
Y_pred = regressor.predict(data_test)
#df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})  
#df

In [39]:
from sklearn import metrics 
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, Y_pred))

Mean Squared Error: 0.5670409553488172
